In [1]:
import pandas as pd

DATA = '/kaggle/input/workout-and-fitness-tracker-data/workout_fitness_tracker_data.csv'
df = pd.read_csv(filepath_or_buffer=DATA, index_col=['User ID'])
df.head()

,Age,Gender,Height (cm),Weight (kg),Workout Type,Workout Duration (mins),Calories Burned,Heart Rate (bpm),Steps Taken,Distance (km),Workout Intensity,Sleep Hours,Water Intake (liters),Daily Calories Intake,Resting Heart Rate (bpm),VO2 Max,Body Fat (%),Mood Before Workout,Mood After Workout
User ID,,,,,,,,,,,,,,,,,,,
1,39,Male,175,99,Cycling,79,384,112,8850,14.44,High,8.2,1.9,3195,61,38.4,28.5,Tired,Fatigued
2,36,Other,157,112,Cardio,73,612,168,2821,1.10,High,8.6,1.9,2541,73,38.4,28.5,Happy,Energized
3,25,Female,180,66,HIIT,27,540,133,18898,7.28,High,9.8,1.9,3362,80,38.4,28.5,Happy,Fatigued
4,56,Male,154,89,Cycling,39,672,118,14102,6.55,Medium,5.8,1.9,2071,65,38.4,28.5,Neutral,Neutral
5,53,Other,194,59,Strength,56,410,170,16518,3.17,Medium,7.3,1.9,3298,59,38.4,28.5,Stressed,Energized


In [2]:
df['Gender'].value_counts().to_dict()

{'Other': 3392, 'Male': 3370, 'Female': 3238}

The fact that we have exactly 10k records, and they are roughly evenly distributed across three genders is an early indication we have synthetic data.

Let's build a scatter plot using dimensionality reduction; this should tell us if our data has a signal in it.

In [3]:
from sklearn.manifold import TSNE

COLUMNS = [key for key, value in df.dtypes.to_dict().items() if str(value) in {'float64', 'int64'}]
RANDOM_STATE = 2025
TARGET = 'Gender'

reducer = TSNE(random_state=RANDOM_STATE)
plot_df = pd.DataFrame(columns=['x', 'y'], data=reducer.fit_transform(X=df[COLUMNS]))
plot_df[TARGET] = df[TARGET].tolist()

In [4]:
from plotly import express
from plotly import io

io.renderers.default = 'iframe'
express.scatter(data_frame=plot_df, x='x', y='y', color=TARGET)

Our data may or may not be synthetic, but whatever signal it contains does not correspond to the target variable, as we do not see clustering in the data such that points in our plot have mostly neighbors with the same value in the target variable.

In [5]:
express.histogram(data_frame=df, x='Height (cm)', facet_col='Gender')

What do we see? First, we see that height is more or less uniformly distributed, which is not what we would expect. Second, we see that women are on average as tall as men, which we would very rarely see in real life in a sample this large.

In [6]:
df[[TARGET, 'Height (cm)']].groupby(by=TARGET).mean().T

Gender,Female,Male,Other
Height (cm),174.332304,174.241246,174.254127


The average woman in our sample is as tall as the average man. That's pretty unlikely in a random sample of 6700 men and women.

In [7]:
express.histogram(data_frame=df, x='Weight (kg)', facet_col='Gender')

Yes. This is synthetic data. Height and weight are not uniformly distributed, especially accounting for sex.

In [8]:
df[[TARGET, 'Weight (kg)']].groupby(by=TARGET).mean().T

Gender,Female,Male,Other
Weight (kg),84.32273,85.525519,84.838443


In [9]:
express.scatter(data_frame=df, x='Height (cm)', y='Weight (kg)', color=TARGET, facet_col=TARGET)

This is definitely synthetic data.

Let's do one more before we go. What does our age pyramid look like?

In [10]:
express.histogram(data_frame=df, x='Age', facet_col=TARGET)

Age counts shouldn't be distributed like this; in a random sample of real data, age is determined by size of birth cohort and mortality/longevity dynamics, and we don't really see either at work here.